In [77]:
# imports
import pandas as pd
import numpy as np
import time

# seed
np.random.seed(0)

In [78]:
# where data is located
!ls ./drive/MyDrive/Thesis\ Workspace/Notebooks/data/set-3
DATA_PATH = "./drive/MyDrive/Thesis Workspace/Notebooks/data/set-3"

dev  test  train


In [79]:
def load_tweet(subfolder):
  df_tweet = pd.read_csv(f'{DATA_PATH}/{subfolder}/tweet.csv').iloc[:, 1:].reset_index(drop=True)
  return df_tweet

subfolder_names = ['dev', 'train', 'test']
df_tweet_dev, df_tweet_train, df_tweet_test = [load_tweet(x) for x in subfolder_names]
for folder, df in zip(subfolder_names, [df_tweet_dev, df_tweet_train, df_tweet_test]):
  print('Folder: ', folder, 'Shape: ', df.shape)
  display(df.head(2))

Folder:  dev Shape:  (401540, 2)


,ID,tweet
0,1224667050301255680,@SparklesOnlyme পুরোনো এইদিনের কথা\n
1,1224667050301255680,@BariraJahan হায়\n


Folder:  train Shape:  (1398465, 2)


,ID,tweet
0,17461978,RT @CarnivalCruise: 🎉 Are you ready to see wha...
1,17461978,Who has time for receipts? Not me. @epson rece...


Folder:  test Shape:  (199863, 2)


,ID,tweet
0,1188812492010487808,RT @clevelanddotcom: Three Ohio House Republic...
1,1188812492010487808,RT @CaliConserv1: California Governor Gavin Ne...


In [80]:
def load_profile_and_label(subfolder):
  df_label = pd.read_csv(f'{DATA_PATH}/{subfolder}/label.csv').iloc[:, 1:]
  df_profile = pd.read_csv(f'{DATA_PATH}/{subfolder}/profile_info.csv').iloc[:, 1:]
  df_return = df_profile.merge(df_label, on='ID')
  return df_return

df_profile_dev, df_profile_train, df_profile_test = [load_profile_and_label(x) for x in subfolder_names]
for folder, df in zip(subfolder_names, [df_profile_dev, df_profile_train, df_profile_test]):
  print('Folder: ', folder, 'Shape: ', df.shape)
  display(df.head(2))

Folder:  dev Shape:  (2365, 36)


,ID,name,screen_name,location,description,url,protected,followers_count,friends_count,listed_count,created_at,favourites_count,utc_offset,time_zone,geo_enabled,verified,statuses_count,lang,contributors_enabled,is_translator,is_translation_enabled,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,label
0,1224667050301255680,Sunny Howlader,sunnyhowlader5,,,NaN,False,136,928,0,Tue Feb 04 12:13:10 +0000 2020,337,NaN,NaN,False,False,99,NaN,False,False,False,F5F8FA,NaN,NaN,False,http://pbs.twimg.com/profile_images/1283246974...,https://pbs.twimg.com/profile_images/128324697...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,0
1,738340405,Hayden Byrne,Maebha_Racing,"Melbourne, Victoria",,NaN,False,2005,4955,12,Sun Aug 05 09:58:22 +0000 2012,1237,NaN,NaN,False,False,3806,NaN,False,False,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg....,False,http://pbs.twimg.com/profile_images/1139046183...,https://pbs.twimg.com/profile_images/113904618...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,1


Folder:  train Shape:  (8278, 36)


,ID,name,screen_name,location,description,url,protected,followers_count,friends_count,listed_count,created_at,favourites_count,utc_offset,time_zone,geo_enabled,verified,statuses_count,lang,contributors_enabled,is_translator,is_translation_enabled,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,label
0,17461978,SHAQ,SHAQ,"Orlando, FL","VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQ...",http://www.ShaqFuRadio.com,False,15349596,692,45568,Tue Nov 18 10:27:25 +0000 2008,142,NaN,NaN,True,True,9798,NaN,False,False,False,080203,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg....,False,http://pbs.twimg.com/profile_images/1673907275...,https://pbs.twimg.com/profile_images/167390727...,2FC2EF,181A1E,252429,666666,True,False,False,False,0
1,1297437077403885568,Jennifer Fishpaw,JenniferFishpaw,,,NaN,False,0,44,0,Sun Aug 23 07:35:11 +0000 2020,7,NaN,NaN,False,False,0,NaN,False,False,False,F5F8FA,NaN,NaN,False,http://pbs.twimg.com/profile_images/1297437406...,https://pbs.twimg.com/profile_images/129743740...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,1


Folder:  test Shape:  (1183, 36)


,ID,name,screen_name,location,description,url,protected,followers_count,friends_count,listed_count,created_at,favourites_count,utc_offset,time_zone,geo_enabled,verified,statuses_count,lang,contributors_enabled,is_translator,is_translation_enabled,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,label
0,1188812492010487808,Sharon Israel ⭐️⭐️⭐️,SharonIsrael10,Los Angeles & Colorado,Day 1 Trump supporter. I rode the escalator! C...,NaN,False,16596,16944,1,Mon Oct 28 13:39:41 +0000 2019,42684,NaN,NaN,False,False,49757,NaN,False,False,False,F5F8FA,NaN,NaN,False,http://pbs.twimg.com/profile_images/1188836420...,https://pbs.twimg.com/profile_images/118883642...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,1
1,155659213,Cristiano Ronaldo,Cristiano,"Turim, Piemonte",This Privacy Policy addresses the collection a...,http://www.facebook.com/cristiano,False,87313765,50,83703,Mon Jun 14 19:09:20 +0000 2010,2,NaN,NaN,True,True,3569,NaN,False,False,True,080101,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg....,False,http://pbs.twimg.com/profile_images/1157313327...,https://pbs.twimg.com/profile_images/115731332...,1643C9,FFFFFF,838387,0D0D0D,True,False,False,False,0


### Regexes
(Tested in `preprocessing-sp15-set3`)

In [81]:
import re

# hashtag #
HASHTAG_REGEX = r"#[a-z0-9_]+"

# mention @
MENTION_REGEX = r"\s([@][\w_-]+)"

# url google.com
URL_REGEX = r"(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"

## Feature Extraction

### User info features

### NOTE
- Cant tell when is the account last updated so here i assumed that the account is updated TODAY
- So age counts from today

In [82]:
def process_user_data(df):
  used_columns = [
        'statuses_count',
        'favourites_count',
        'listed_count',
        'label'
    ]

  df_return = df[used_columns]

  # temporary use today as the 'Age milestone' for accounts
  age = (np.datetime64('today') - pd.to_datetime(df['created_at']).dt.tz_localize(None)) / np.timedelta64(1, 'Y')
  df_return['age'] = age
  df_return['follower_to_friend_ratio'] = df['followers_count']/df['friends_count']
  df_return['tweet_frequency'] = df['statuses_count']/age #check, perday or what?
  df_return['ID'] = df['ID']
  return df_return

In [83]:
feature_time = time.time()
extracted_df_profile_dev = process_user_data(df_profile_dev)
end_feature_time = time.time()
print('Time: ', end_feature_time - feature_time)
print('shape: ', extracted_df_profile_dev.shape)
extracted_df_profile_dev.head()

Time:  0.4780309200286865
shape:  (2365, 8)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

,statuses_count,favourites_count,listed_count,label,age,follower_to_friend_ratio,tweet_frequency,ID
0,99,337,0,0,1.915141,0.146552,51.693324,1224667050301255680
1,3806,1237,12,1,9.414525,0.404642,404.268957,738340405
2,36952,28306,939,0,11.937730,20.335921,3095.395798,108701392
3,24546,31,2419,1,14.147555,8.399891,1734.999390,10156532
4,301967,1051,17230,0,13.120160,9.796653,23015.497227,17525171


In [84]:
feature_time = time.time()
extracted_df_profile_train = process_user_data(df_profile_train)
end_feature_time = time.time()
print('Time: ', end_feature_time - feature_time)
print('shape: ', extracted_df_profile_train.shape)
extracted_df_profile_train.head()

Time:  1.616631269454956
shape:  (8278, 8)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

,statuses_count,favourites_count,listed_count,label,age,follower_to_friend_ratio,tweet_frequency,ID
0,9798,142,45568,0,13.127071,22181.497110,746.396501,17461978
1,0,7,0,1,1.365350,0.000000,0.000000,1297437077403885568
2,5518,953,3201,0,13.101479,1605.976842,421.173819,17685258
3,192876,2946,1744,0,13.411272,68.233076,14381.633489,15750898
4,103,729,44,1,8.402246,20.593509,12.258627,1659167666


In [85]:
feature_time = time.time()
extracted_df_profile_test = process_user_data(df_profile_test)
end_feature_time = time.time()
print('Time: ', end_feature_time - feature_time)
print('shape: ', extracted_df_profile_test.shape)
extracted_df_profile_test.head()

Time:  0.256150484085083
shape:  (1183, 8)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

,statuses_count,favourites_count,listed_count,label,age,follower_to_friend_ratio,tweet_frequency,ID
0,49757,42684,1,1,2.186029,9.794618e-01,22761.360948,1188812492010487808
1,3569,2,83703,0,11.557258,1.746275e+06,308.810270,155659213
2,73786,762,1471,0,11.614409,4.482742e+02,6352.970924,147725246
3,2,0,0,1,1.374328,1.657459e-02,1.455257,1296248637194895360
4,11945,1123,40146,0,8.737337,3.370332e+04,1367.121352,1339835893


### Tweet Features

### NOTE:
Missing features related to tweet interaction
- favorite_received
- retweet_received
- source_count

In [86]:
def process_tweet_data(df):
  tweet_series = df['tweet'].astype(str)
  df = df.assign(num_urls = tweet_series.apply(lambda x: len(re.findall(URL_REGEX, x))))
  df_grouped = df.groupby('ID')
  df_return = pd.DataFrame()
  # tweet_count = df_grouped['ID'].count()
  # favorite_received = df_grouped['favorite_count'].sum()
  # df_return['favorite_received'] = favorite_received
  # df_return['favorite_received_ratio'] = favorite_received / tweet_count

  # retweet_received = df_grouped['retweet_count'].sum()
  # df_return['retweet_received'] = retweet_received
  # df_return['retweet_received_ratio'] = retweet_received / tweet_count

  df_return['url_count'] = df_grouped['num_urls'].sum()
  # df_return['activity_source_count'] = df_grouped['source'].nunique()
  return df_return

In [87]:
feature_time = time.time()
extracted_df_tweet_dev = process_tweet_data(df_tweet_dev)
end_feature_time = time.time()
print('Time: ', end_feature_time - feature_time)
print('shape: ', extracted_df_tweet_dev.shape)
extracted_df_tweet_dev.head()

Time:  10.069056034088135
shape:  (2365, 1)


,url_count
ID,
3382,47
13348,43
759251,224
813286,161
994431,5


In [88]:
feature_time = time.time()
extracted_df_tweet_train = process_tweet_data(df_tweet_train)
end_feature_time = time.time()
print('Time: ', end_feature_time - feature_time)
print('shape: ', extracted_df_tweet_train.shape)
extracted_df_tweet_train.head()

Time:  33.805113554000854
shape:  (8278, 1)


,url_count
ID,
12,78
1605,50
10350,45
10441,120
12830,246


In [89]:
feature_time = time.time()
extracted_df_tweet_test = process_tweet_data(df_tweet_test)
end_feature_time = time.time()
print('Time: ', end_feature_time - feature_time)
print('shape: ', extracted_df_tweet_test.shape)
extracted_df_tweet_test.head()

Time:  4.876970529556274
shape:  (1183, 1)


,url_count
ID,
691353,107
1068831,199
1367531,200
5286532,50
5894372,184


### Merge
**Also drop ID**

In [98]:
zipped_output = zip(
    [extracted_df_profile_dev, extracted_df_profile_train, extracted_df_profile_test], 
    [extracted_df_tweet_dev, extracted_df_tweet_train, extracted_df_tweet_test]
)
df_dev, df_train, df_test = [ pd.merge(user, tweet, on='ID').drop('ID', axis = 1) for user, tweet in zipped_output]
for df in df_dev, df_train, df_test:
  print(df.shape)
  display(df.head(2))

(2365, 8)


,statuses_count,favourites_count,listed_count,label,age,follower_to_friend_ratio,tweet_frequency,url_count
0,99,337,0,0,1.915141,0.146552,51.693324,19
1,3806,1237,12,1,9.414525,0.404642,404.268957,20


(8278, 8)


,statuses_count,favourites_count,listed_count,label,age,follower_to_friend_ratio,tweet_frequency,url_count
0,9798,142,45568,0,13.127071,22181.49711,746.396501,207
1,0,7,0,1,1.365350,0.00000,0.000000,0


(1183, 8)


,statuses_count,favourites_count,listed_count,label,age,follower_to_friend_ratio,tweet_frequency,url_count
0,49757,42684,1,1,2.186029,9.794618e-01,22761.360948,55
1,3569,2,83703,0,11.557258,1.746275e+06,308.810270,216


### Check and Cleanup

In [99]:
outputs = [df_dev, df_train, df_test]

In [100]:
for df in outputs:
  display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2365 entries, 0 to 2364
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   statuses_count            2365 non-null   int64  
 1   favourites_count          2365 non-null   int64  
 2   listed_count              2365 non-null   int64  
 3   label                     2365 non-null   int64  
 4   age                       2365 non-null   float64
 5   follower_to_friend_ratio  2365 non-null   float64
 6   tweet_frequency           2365 non-null   float64
 7   url_count                 2365 non-null   int64  
dtypes: float64(3), int64(5)
memory usage: 166.3 KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8278 entries, 0 to 8277
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   statuses_count            8278 non-null   int64  
 1   favourites_count          8278 non-null   int64  
 2   listed_count              8278 non-null   int64  
 3   label                     8278 non-null   int64  
 4   age                       8278 non-null   float64
 5   follower_to_friend_ratio  8278 non-null   float64
 6   tweet_frequency           8278 non-null   float64
 7   url_count                 8278 non-null   int64  
dtypes: float64(3), int64(5)
memory usage: 582.0 KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1183 entries, 0 to 1182
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   statuses_count            1183 non-null   int64  
 1   favourites_count          1183 non-null   int64  
 2   listed_count              1183 non-null   int64  
 3   label                     1183 non-null   int64  
 4   age                       1183 non-null   float64
 5   follower_to_friend_ratio  1183 non-null   float64
 6   tweet_frequency           1183 non-null   float64
 7   url_count                 1183 non-null   int64  
dtypes: float64(3), int64(5)
memory usage: 83.2 KB


None

In [101]:
for df in outputs:
  display(df.apply(lambda x: x.unique()))

statuses_count              [99, 3806, 36952, 24546, 301967, 44012, 15, 23...
favourites_count            [337, 1237, 28306, 31, 1051, 204694, 55, 1240,...
listed_count                [0, 12, 939, 2419, 17230, 3, 18065, 3612, 2261...
label                                                                  [0, 1]
age                         [1.9151409172850407, 9.414524507943606, 11.937...
follower_to_friend_ratio    [0.14655172413793102, 0.4046417759838547, 20.3...
tweet_frequency             [51.693324029829235, 404.2689566306452, 3095.3...
url_count                   [19, 20, 201, 196, 214, 24, 1, 87, 250, 115, 1...
dtype: object

statuses_count              [9798, 0, 5518, 192876, 103, 4879, 31055, 1873...
favourites_count            [142, 7, 953, 2946, 729, 177, 4532, 27, 0, 59,...
listed_count                [45568, 0, 3201, 1744, 44, 26952, 550181, 5279...
label                                                                  [0, 1]
age                         [13.127071175948805, 1.3653501453499057, 13.10...
follower_to_friend_ratio    [22181.49710982659, 0.0, 1605.9768421052631, 6...
tweet_frequency             [746.3965014489848, 0.0, 421.17381905559506, 1...
url_count                   [207, 0, 95, 173, 57, 178, 112, 191, 1, 135, 1...
dtype: object

statuses_count              [49757, 3569, 73786, 2, 11945, 37970, 223627, ...
favourites_count            [42684, 2, 762, 0, 1123, 30750, 6720, 6535, 86...
listed_count                [1, 83703, 1471, 0, 40146, 29, 18101, 17156, 3...
label                                                                  [1, 0]
age                         [2.186029214735314, 11.557258128098049, 11.614...
follower_to_friend_ratio    [0.9794617563739377, 1746275.3, 448.2742382271...
tweet_frequency             [22761.36094824543, 308.8102697406259, 6352.97...
url_count                   [55, 216, 197, 0, 163, 76, 201, 288, 198, 81, ...
dtype: object

In [102]:
for df in outputs:
  display(df.isin([np.nan, np.inf, -np.inf]).sum())

statuses_count               0
favourites_count             0
listed_count                 0
label                        0
age                          0
follower_to_friend_ratio    16
tweet_frequency              0
url_count                    0
dtype: int64

statuses_count               0
favourites_count             0
listed_count                 0
label                        0
age                          0
follower_to_friend_ratio    71
tweet_frequency              0
url_count                    0
dtype: int64

statuses_count               0
favourites_count             0
listed_count                 0
label                        0
age                          0
follower_to_friend_ratio    13
tweet_frequency              0
url_count                    0
dtype: int64

### Fill Na `0`

In [103]:
df_dev = df_dev.replace([np.inf, -np.inf], np.nan).fillna(0)
df_train = df_train.replace([np.inf, -np.inf], np.nan).fillna(0)
df_test = df_test.replace([np.inf, -np.inf], np.nan).fillna(0)

In [104]:
for df in [df_dev, df_train, df_test]:
  display(df.isin([np.nan, np.inf, -np.inf]).sum())

statuses_count              0
favourites_count            0
listed_count                0
label                       0
age                         0
follower_to_friend_ratio    0
tweet_frequency             0
url_count                   0
dtype: int64

statuses_count              0
favourites_count            0
listed_count                0
label                       0
age                         0
follower_to_friend_ratio    0
tweet_frequency             0
url_count                   0
dtype: int64

statuses_count              0
favourites_count            0
listed_count                0
label                       0
age                         0
follower_to_friend_ratio    0
tweet_frequency             0
url_count                   0
dtype: int64

### Save

In [105]:
save_path = "./drive/MyDrive/Thesis Workspace/Notebooks/data/sb14-set3/"
for name, df in zip(subfolder_names, [df_dev, df_train, df_test]):
  df.to_csv(f'{save_path}{name}.csv', index = False)